# client

> Fill in a module description here

In [ ]:
#| default_exp client

In [1]:
#| hide
from nbdev.showdoc import *

ModuleNotFoundError: ignored

In [4]:
#| export
import os
import http.client
import json
from typing import Any, Dict

In [3]:
#| export
class GraphQLClient:
  def __init__(self, host: str, path: str = "/") -> None:
    self._host = host
    self._path = path

  def __call__(self, query: str, variables: Dict[str, Any]={}, host: str = None, path: str = None):
    payload = {
      "query": query,
      "variables": variables,
    }

    conn = http.client.HTTPSConnection(self._host)
    conn.request("POST", self._path, json.dumps(payload), {
      "Content-Type": "application/json",
      "Authorization": "Bearer " + os.environ['METERED_API_KEY']
    })
    res = conn.getresponse()
    data = res.read()
    json_data = json.loads(data.decode("utf-8"))

    if "errors" in json_data and json_data["errors"]:
      raise Exception(json.dumps(json_data["errors"]))
    
    return json_data


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()